# What is Autoencoders

Autoencoders is type of neual network used to learn efficient data codings in an unsupervised manner. The aim of an autoencoder is to learn a representation (encoding) for a set of data.

In [ ]:
!pip install imutils



# Libraries


In [ ]:
import os
import glob
import numpy as np 
import pandas as pd 
import os
from os import listdir
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import imutils    

from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization,Flatten,Activation,Dense,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle #shuffling the data improves the model
from keras.preprocessing import image
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout, UpSampling2D


# Image Dir settings


In [ ]:
image_dir="../input/brain-mri-images-for-brain-tumor-detection/"

In [ ]:
image_dir

In [ ]:
plt.figure(figsize=(20,8))
img = cv2.imread('../input/brain-mri-images-for-brain-tumor-detection/yes/Y1.jpg', 0)
plt.imshow(img, cmap='gray')
print(img.shape)

In [ ]:
Yes_IMAGES = glob.glob('../input/brain-mri-images-for-brain-tumor-detection/yes/*.jpg')
No_IMAGES = glob.glob('../input/brain-mri-images-for-brain-tumor-detection/no/*.jpg')

In [ ]:
def load_image(path):
    image_list = np.zeros((len(path), 258, 540, 1))
    for i, fig in enumerate(path):
        img = image.load_img(fig, color_mode='grayscale', target_size=(258, 540))
        x = image.img_to_array(img).astype('float32')
        x = x / 255.0
        image_list[i] = x
    
    return image_list



# Dataset Spliting


In [ ]:
x_train = load_image(Yes_IMAGES)
y_train = load_image(Yes_IMAGES)
x_test = load_image(No_IMAGES)

print(x_train.shape, x_test.shape)

In [ ]:
def train_val_split(x_train, y_train):
    rnd = np.random.RandomState(seed=42)
    perm = rnd.permutation(len(x_train))
    train_idx = perm[:int(0.8 * len(x_train))]
    val_idx = perm[int(0.8 * len(x_train)):]
    return x_train[train_idx], y_train[train_idx], x_train[val_idx], y_train[val_idx]

x_train, y_train, x_val, y_val = train_val_split(x_train, y_train)
print(x_train.shape, x_val.shape)


# Creating Autoencoder Class


In [ ]:
class Autoencoder():
    def __init__(self):
        self.img_rows = 258
        self.img_cols = 540
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        optimizer = Adam(lr=0.001)
        
        self.autoencoder_model = self.build_model()
        self.autoencoder_model.compile(loss='mse', optimizer=optimizer)
        self.autoencoder_model.summary()
    
    def build_model(self):
        input_layer = Input(shape=self.img_shape)
        
        # encoder
        h = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
        h = MaxPooling2D((2, 2), padding='same')(h)
        
        # decoder
        h = Conv2D(64, (3, 3), activation='relu', padding='same')(h)
        h = UpSampling2D((2, 2))(h)
        output_layer = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(h)
        
        return Model(input_layer, output_layer)
    
    def train_model(self, x_train, y_train, x_val, y_val, epochs, batch_size=20):
        early_stopping = EarlyStopping(monitor='val_loss',
                                       min_delta=0,
                                       patience=5,
                                       verbose=1, 
                                       mode='auto')
        history = self.autoencoder_model.fit(x_train, y_train,
                                             batch_size=batch_size,
                                             epochs=epochs,
                                             validation_data=(x_val, y_val),
                                             callbacks=[early_stopping])
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()
    
    def eval_model(self, x_test):
        preds = self.autoencoder_model.predict(x_test)
        return preds



# Model Training


In [ ]:
ae = Autoencoder()
ae.train_model(x_train, y_train, x_val, y_val, epochs=200, batch_size=20)



# Predictions


In [ ]:
preds = ae.eval_model(x_test)

In [ ]:
preds_0 = preds[10] * 255.0
preds_0 = preds_0.reshape(258, 540)
x_test_0 = x_test[10] * 255.0
x_test_0 = x_test_0.reshape(258, 540)
plt.imshow(x_test_0, cmap='gray')

In [ ]:
plt.imshow(preds_0, cmap='gray')

In [ ]:
plt.imshow(preds_0, cmap='bone')

In [ ]:
preds = preds.reshape(-1, 258, 540)

In [ ]:
preds

In [ ]:
preds_1 = preds[20] * 255.0
preds_1 = preds_1.reshape(258, 540)

plt.imshow(preds_1, cmap='PiYG')

In [ ]:
preds_2 = preds[50] * 255.0
preds_2 = preds_2.reshape(258, 540)

plt.imshow(preds_1, cmap='twilight')